In [9]:
import array
import csv, json
import math
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
from PIL import Image

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

def getSmallestAbsoluteKey(item):
    #return min(abs(item[3]), abs(item(4)))
    return min(abs(item[3]),abs(item[4]))

In [10]:
data = []
with open("obesity.csv", "rb") as csvfile:
    reader = csv.DictReader(csvfile)
    for line in reader:
        data.append(line)


In [11]:
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))
hex_colors = ['#ffffff','#fffefe','#fffcfd','#fffcfd','#fffafc','#fff9fc','#fff7fa','#fff7fa','#fff6f9','#fff5f8','#fff3f8','#fff2f7','#fff1f6','#ffeff5','#ffeef5','#ffedf4','#ffebf3','#ffeaf2','#ffeaf2','#ffe9f1','#ffe7f0','#ffe6ef','#ffe4ef','#ffe4ee','#ffe2ed','#ffe1ec','#ffe0ec','#ffdfeb','#ffddea','#ffdcea','#ffdbe9','#ffd9e8','#ffd8e7','#ffd8e7','#ffd5e6','#ffd4e5','#ffd3e5','#ffd3e4','#ffd0e3','#ffcfe2','#ffcfe2','#ffcde1','#ffcbe0','#ffcadf','#ffc9df','#ffc9de','#ffc6dd','#ffc6dc','#ffc4dc','#ffc3db','#ffc2da','#ffc1da','#ffc0d9','#ffbed8','#ffbcd7','#ffbbd7','#ffbad6','#ffb9d5','#ffb8d5','#ffb6d4','#ffb4d3','#ffb4d2','#feb2d2','#feb1d1','#feb1d1','#fdafd0','#fdaecf','#fdadcf','#fdabce','#fcaace','#fca9cd','#fca8cc','#fba7cb','#fba5cb','#fba4ca','#faa3ca','#faa2c9','#f9a1c8','#f9a0c8','#f99ec7','#f89ec7','#f89dc6','#f89bc5','#f799c5','#f798c4','#f697c4','#f697c3','#f596c3','#f594c2','#f593c1','#f492c1','#f390c0','#f38fbf','#f38ebf','#f28ebf','#f28cbe','#f18bbd','#f08abd','#f089bc','#ef87bb','#ef87bb','#ee86bb','#ee84ba','#ee84ba','#ed82b9','#ec80b8','#eb80b8','#eb7fb7','#ea7db7','#ea7cb6','#e97cb6','#e87ab5','#e879b4','#e778b4','#e776b3','#e676b3','#e574b2','#e574b2','#e472b1','#e371b1','#e370b0','#e370b0','#e16eaf','#e16daf','#e06cae','#df6bae','#df69ad','#de68ad','#dd67ac','#dd66ac','#dc66ab','#db64ab','#da63aa','#d962aa','#d961aa','#d860a9','#d85fa8','#d75da8','#d65ca7','#d55ca7','#d45aa7','#d459a6','#d359a6','#d257a5','#d156a5','#d155a4','#cf54a4','#cf53a4','#ce53a3','#cd52a3','#cc50a2','#cb4fa2','#ca4ea1','#ca4da1','#c84ca0','#c84ba0','#c74aa0','#c6499f','#c5489f','#c4479e','#c3469e','#c3459d','#c1449d','#c1429d','#c0429c','#bf419c','#be409b','#bd3f9b','#bc3d9b','#bb3d9a','#ba3c9a','#b93b99','#b83a99','#b73999','#b63898','#b63798','#b43698','#b43597','#b23497','#b13397','#b03296','#b03196','#ae3096','#ad2f95','#ac2e95','#ab2d94','#aa2c94','#a82b93','#a82a93','#a72993','#a52893','#a52792','#a42692','#a22592','#a12591','#a02491','#9f2291','#9e2290','#9d2090','#9c2090','#9a1f8f','#9a1e8f','#981d8f','#971c8e','#961b8e','#951a8e','#93198e','#93188d','#91188d','#8f168d','#8e158c','#8e158c','#8c138c','#8b128b','#89128b','#88118b','#87108b','#850f8a','#840e8a','#830d8a','#810c8a','#810b89','#7f0a89','#7e0989','#7c0889','#7b0888','#790788','#790688','#770588','#760587','#750487','#720387','#710387','#700286','#6e0286','#6c0286','#6c0186','#690185','#680185','#660085','#650085','#640085','#620084','#600084','#5f0084','#5c0084','#5a0084','#5a0083','#580083','#560083','#540083','#520083','#510082','#4e0082','#4d0082','#4b0082']

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i]) # set the colour accordingly
img.show
img.save("color-map.png")

In [12]:
# Include RCP
# Get just chg_cap
obesity = {}
minval = 0.011
maxval = 0.601
for row in data:
    country = row['location']
    year = row["year"]
    metric = row["unit"]
    value = float(row["mean"])
    if country.isalpha() and country != "G":
        if not obesity.has_key(country):
            obesity[country] = []
            for y in range(1990, 2014):
                val_0 = {"year": y, "pct": 0}
                obesity[country].append(val_0)
        years_idx = int(year)%1990
        obesity[country][years_idx][metric] = (value - minval)/(maxval - minval)

In [13]:
cow = []
with open("cow.csv", "rb") as csvfile:
    reader = csv.DictReader(csvfile,delimiter=";")
    for line in reader:
        cow.append(line)
centroids = {}
for row in cow:
    idx = row['ISO3166A3'].strip()
    lng = float(row['longitude'].strip())
    lat = float(row['latitude'].strip())
    if not centroids.has_key(idx):
        centroids[idx] = [lng,lat]

centroids

{'ABW': [-69.9666666667, 12.5],
 'AFG': [65.0, 33.0],
 'AGO': [18.5, -12.5],
 'AIA': [-63.1666666667, 18.25],
 'ALA': [20.0, 60.25],
 'ALB': [20.0, 41.0],
 'AND': [1.5, 42.5],
 'ARE': [54.0, 24.0],
 'ARG': [-64.0, -34.0],
 'ARM': [45.0, 40.0],
 'ASM': [-170.0, -14.3333333333],
 'ATA': [0.0, -90.0],
 'ATF': [77.5333333333, -37.8333333333],
 'ATG': [-61.8, 17.05],
 'AUS': [133.0, -27.0],
 'AUT': [13.3333333333, 47.3333333333],
 'AZE': [47.5, 40.5],
 'BDI': [30.0, -3.5],
 'BEL': [4.0, 50.8333333333],
 'BEN': [2.25, 9.5],
 'BES': [-68.25, 12.1666666667],
 'BFA': [-2.0, 13.0],
 'BGD': [90.0, 24.0],
 'BGR': [25.0, 43.0],
 'BHR': [50.55, 26.0],
 'BHS': [-76.0, 24.25],
 'BIH': [18.0, 44.0],
 'BLM': [-62.833333, 17.9],
 'BLR': [28.0, 53.0],
 'BLZ': [-88.75, 17.25],
 'BMU': [-64.75, 32.3333333333],
 'BOL': [-65.0, -17.0],
 'BRA': [-55.0, -10.0],
 'BRB': [-59.5333333333, 13.1666666667],
 'BRN': [114.666666667, 4.5],
 'BTN': [90.5, 27.5],
 'BVT': [3.4, -54.4333333333],
 'BWA': [24.0, -22.0],
 'CAF

In [14]:
points = []
for country_name in obesity.keys():
    country = obesity[country_name]
    x,y = LonLatToPixelXY(centroids[country_name])
    years = obesity[country_name]
    for i in range(0,len(years)-1):
        year = years[i]["year"]
        val_1 = years[i]["pct"]
        val_2 = years[i+1]["pct"]
        points += [x,y,year,val_1,val_2]
    year = years[i+1]["year"]
    val_1 = years[i+1]["pct"]
    val_2 = years[i+1]["pct"]
    points += [x,y,year,val_1,val_2]
    
split = split_list(points,len(points)/5)
ordered_split = sorted(split, key=getSmallestAbsoluteKey, reverse=True)
points = []
for x in ordered_split:
    for y in x:
        points += [y]

array.array('f', points).tofile(open("obesity-centroids.bin", 'wb'))